# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

from scipy import stats
import statsmodels.stats.power as smp

from math import sqrt, ceil

from sympy import *

# Формулы

## Задаем пространство символов

In [102]:
var('p_c p_t MDE z1_alpha z1_beta q x')

(p_c, p_t, MDE, z1_alpha, z1_beta, q, x)

## Задаем формулы

In [39]:
effect_size_expr = MDE/sqrt(p_t*(1-p_t)+(p_c*(1-p_c)))

effect_size_expr

MDE/sqrt(p_c*(1 - p_c) + p_t*(1 - p_t))

In [40]:
q_expression = sqrt(p_t*(1-p_t))/(sqrt(p_c*(1-p_c))+sqrt(p_t*(1-p_t)))

q_expression

sqrt(p_t*(1 - p_t))/(sqrt(p_c*(1 - p_c)) + sqrt(p_t*(1 - p_t)))

In [41]:
n_expression = (((z1_alpha + z1_beta)**2)*(p_t*(1-p_t)/q + p_c*(1-p_c)/(1-q)))/MDE**2

n_expression

(z1_alpha + z1_beta)**2*(p_c*(1 - p_c)/(1 - q) + p_t*(1 - p_t)/q)/MDE**2

# Задача 1 (на опредение объема выборки)
___________

Вы работаете в интернет-магазине и хотите провести A/B тест для нового дизайна страницы товара, чтобы определить, какой из дизайонов приводит к большему числу покупок. Из предыдущик тестов известно, что **конверсия** на текущем дизайне составляет около **5%**. Вы хотите обнаружить изменение **конверсии на 1%**, имея уровень **значимости 95% и мощность теста 80%**. Какой объем выборки необходим для каждой группы (A и B)? 

In [46]:
p_c_num = 0.05
p_t_num = 0.06
MDE_num = round(p_t_num - p_c_num, 2)
alpha = 0.05
power = 0.8
beta = round(1 - power, 2)
effect_size = float(effect_size_expr.evalf(subs={p_t: p_t_num, 
                                           p_c:p_c_num,
                                           MDE:MDE_num}))

z_1_minus_alpha = stats.norm.ppf(1-alpha)
z_1_minus_beta = stats.norm.ppf(1-beta)

In [47]:
z_1_minus_alpha

1.6448536269514722

In [48]:
z_1_minus_beta

0.8416212335729143

## Пакет stats

In [49]:
# chipower = smp.GofChisquarePower()
# size_of_each_test_group = int(chipower.solve_power(
#                                          effect_size=effect_size,
#                                          nobs=None,
#                                          alpha=alpha,
#                                          power=power,
#                                          n_bins=2))

In [61]:
size_of_each_test_group = smp.zt_ind_solve_power(effect_size=effect_size, nobs1=None, 
                                                                     alpha=alpha, power=power, ratio=1, 
                                                                     alternative='larger')

In [62]:
print(f'Необходимый размер выборки для эксперимента - {size_of_each_test_group}')

Необходимый размер выборки для эксперимента - 12847.353928137107


## Вручную

In [63]:
q_num = q_expression.evalf(subs={p_t: p_t_num, 
                             p_c:p_c_num})

q_num

0.521454249614592

In [64]:
n_num = n_expression.evalf(subs={p_t: p_t_num, 
                                 p_c: p_c_num, 
                                 z1_alpha: z_1_minus_alpha, 
                                 z1_beta: z_1_minus_beta,
                                 q: q_num,
                                 MDE: MDE_num})

In [65]:
print(f'Необходимый размер выборки для эксперимента - {int(n_num)}')

Необходимый размер выборки для эксперимента - 12823


In [66]:
n_num * q_num

6686.99561166943

In [67]:
n_num * (1-q_num)

6136.74801801967

# Задача 2 (на определение длительности эксперимента)
___________
Вы разрабатываете новое приложение для здорового образа жизни и планируете провести A/B тестирование двух различных функций: A - ежедневные уведомления с советами по здоровому питанию, B - трекер физической активности. Вы хотите определить, какая функция приведет к большему уровню удержания пользователей на **10%** относительно текущего уровня удержания в **40%**. При текущем числе пользователей в **5000**, какой период времени вам понадобится для проведения теста, чтобы получить статистически значимые результаты с уровнем значимости **90%**?

In [80]:
p_c_num = 0.4
p_t_num = 0.5
MDE_num = round(p_t_num - p_c_num, 2)
alpha = 0.1
effect_size = float(effect_size_expr.evalf(subs={p_t: p_t_num, 
                                           p_c:p_c_num,
                                           MDE:MDE_num}))
power = 0.8
beta = round(1 - power, 2)

z_1_minus_alpha = stats.norm.ppf(1-alpha)
z_1_minus_beta = stats.norm.ppf(1-beta)

In [81]:
z_1_minus_alpha

1.2815515655446004

In [82]:
z_1_minus_beta

0.8416212335729143

## Пакет stats

In [83]:
size_of_each_test_group = smp.zt_ind_solve_power(effect_size=effect_size, nobs1=None, 
                                                                     alpha=alpha, power=power, ratio=1.0, 
                                                                     alternative='larger')

In [85]:
# size_of_each_test_group = int(chipower.solve_power(
#                                          effect_size=effect_size,
#                                          nobs=None,
#                                          alpha=alpha*2,
#                                          power=power,
#                                          n_bins=2))

In [87]:
print(f'Необходимый размер выборки для эксперимента - {size_of_each_test_group}')

Необходимый размер выборки для эксперимента - 441.770548021326


In [93]:
days = ceil(5000/(size_of_each_test_group))
days

12

## Вручную

In [89]:
q_num = q_expression.evalf(subs={p_t: p_t_num, 
                             p_c:p_c_num})

q_num

0.505102572168219

In [90]:
n_num = n_expression.evalf(subs={p_t: p_t_num, 
                                 p_c: p_c_num, 
                                 z1_alpha: z_1_minus_alpha, 
                                 z1_beta: z_1_minus_beta,
                                 q: q_num,
                                 MDE: MDE_num})

n_num

441.724544631567

In [91]:
print(f'Необходимый размер выборки для эксперимента - {int(n_num)}')

Необходимый размер выборки для эксперимента - 441


In [92]:
days = ceil(5000/(n_num))
days

12

# Задача 3 (на определение выборки при различных сегментах)
_______

Вы работаете в маркетинговом агентстве и хотите провести A/B тестирование двух различных рекламных кампаний (A и B) на целевую аудиторию. Однако у вас есть несколько различных сегментов целевой аудитории с разными размерами: **сегмент 1 (1000 пользователей), сегмент 2 (5000 пользователей), сегмент 3 (30000 пользователей)**. Вы хотите обнаружить **изменение конверсии на 2%** с уровнем **значимости 95%** и **мощностью теста 80%** для каждого сегмента. Сколько пользователей необходимо включить в каждый из сегментов (A и B)?

In [119]:
seg_1 = 1000
seg_2 = 5000
seg_3 = 30000
N = seg_1 + seg_2 + seg_3

seg_1_perc = round(seg_1 / N, 2)
seg_2_perc = round(seg_2 / N, 2)
seg_3_perc = round(seg_3 / N, 2)

p_c_num = 0.5
p_t_num = 0.52
MDE_num = round(p_t_num - p_c_num, 2)

alpha = 0.05
effect_size = float(effect_size_expr.evalf(subs={p_t: p_t_num, 
                                           p_c:p_c_num,
                                           MDE:MDE_num}))
power = 0.8
beta = round(1 - power, 2)

z_1_minus_alpha = stats.norm.ppf(1-(alpha/2))
z_1_minus_beta = stats.norm.ppf(1-beta)

In [120]:
z_1_minus_alpha

1.959963984540054

In [121]:
z_1_minus_beta

0.8416212335729143

## Пакет stats

In [122]:
size_of_each_test_group = smp.zt_ind_solve_power(effect_size=effect_size, nobs1=None, 
                                                                     alpha=alpha, power=power, ratio=1.0, 
                                                                     alternative='two-sided')

print(f'Необходимый размер выборки для эксперимента - {int(size_of_each_test_group)}')

Необходимый размер выборки для эксперимента - 19606


## Вручную

In [123]:
q_num = q_expression.evalf(subs={p_t: p_t_num, 
                                 p_c:p_c_num})

q_num

0.499799839839821

In [124]:
n_num = n_expression.evalf(subs={p_t: p_t_num, 
                                 p_c: p_c_num, 
                                 z1_alpha: z_1_minus_alpha, 
                                 z1_beta: z_1_minus_beta,
                                 q: q_num,
                                 MDE: MDE_num})

print(f'Необходимый размер выборки для эксперимента - {int(n_num)}')

Необходимый размер выборки для эксперимента - 19606


## Считаем по сегментам

In [132]:
sample_seg_1 = int(n_num*seg_1_perc)
sample_seg_2 = int(n_num*seg_2_perc)
sample_seg_3 = int(n_num*seg_3_perc)

print(f'Сегмент 1: {sample_seg_1}\nСегмент 2: {sample_seg_2}\nСегмент 3: {sample_seg_3}')

Сегмент 1: 588
Сегмент 2: 2744
Сегмент 3: 16273
